This file is used to build all of the json data that gets loaded by `train.ipynb`

In [ ]:
# Create palette file from palette.png
from util.convert_image_to_palette import convert 

convert("data/palette.png", "data/palette.json")

In [ ]:
# Convert all files in the training .vox folder to json files
import os
from util.convert_vox_to_json import convert_vox_to_json

filenames = os.listdir('training/vox')

for filename in filenames:
    if "vox" in filename:
        new_filename = filename.replace("vox", "json")
        convert_vox_to_json("training/vox/" + filename, "training/json/" + new_filename)

In [20]:
# Convert training json files to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('training/json')

for filename in filenames:
    print(filename)
    if "json" in filename:
        new_filename = filename.replace("json", "vox")
        convert_json_to_vox("training/json/" + filename, "training/view_vox/" + new_filename)

book.json
flag.json
fountain.json
grapes.json
grass.json
juice.json
pearl.json
pine.json
rocks.json
sorbub.json
sordype.json
sorjek.json
sorpok.json
spool.json
tree.json
wall.json
_exp_book_32478025969.json
_exp_book_36924284346.json
_exp_book_39294823921.json
_exp_book_60428634015.json
_exp_book_63864055455.json
_exp_book_65664084818.json
_exp_book_67669236979.json
_exp_book_77124027583.json
_exp_book_77297708260.json
_exp_book_85184400841.json
_exp_book_86559097141.json
_exp_book_99290167339.json
_exp_flag_21679507102.json
_exp_flag_41988750724.json
_exp_flag_5452156123.json
_exp_flag_56200643847.json
_exp_flag_6315030431.json
_exp_flag_64121523933.json
_exp_flag_72343915713.json
_exp_flag_74193493035.json
_exp_flag_97547514738.json
_exp_fountain_13954074278.json
_exp_fountain_1579708987.json
_exp_fountain_17651450779.json
_exp_fountain_17784610806.json
_exp_fountain_2061494971.json
_exp_fountain_26812843922.json
_exp_fountain_33289814988.json
_exp_fountain_37995143093.json
_exp_foun

In [ ]:
# Convert json files in examples to vox files for viewing
import os
from util.convert_vox_to_json import convert_json_to_vox

filenames = os.listdir('examples/json')

for filename in filenames:
    if "json" in filename:
        new_filename = filename.replace("json", "vox")
        convert_json_to_vox("examples/json/" + filename, "examples/view_vox/" + new_filename)

In [ ]:
# Delete all example files
import os

json_dir = "examples/json/"
json_files = os.listdir(json_dir)

vox_dir = "examples/view_vox/"
vox_files = os.listdir(vox_dir)

for item in json_files:
    if item.endswith(".json"):
        os.remove(os.path.join(json_dir, item))

for item in vox_files:
    if item.endswith(".vox"):
        os.remove(os.path.join(vox_dir, item))

In [ ]:
import training
import vector as vec
import random
import json
import os

# Delete existing sorpok files
sorpok_dir = "training/json/"
sorpok_files = os.listdir(sorpok_dir)

for item in sorpok_files:
    if "sorpok" in item:
        os.remove(os.path.join(sorpok_dir, item))

# Generate sorpok files
sorpok_count = 50

# Light Blue: 246
# Blue: 231
# Yellow: 252
# Green: 47
# White: 15
# Red: 192

# Set up parameters
for i in range(sorpok_count):
    size = 4
    r = random.random()
    if r < 0.5:
        color1 = 246
        color2 = 231
    else:
        color1 = 15
        color2 = 192
    offset = (int(random.random() * size), int(random.random() * size), int(random.random() * size))
    data = {
        "size": {
            "x": training.SIZE[0],
            "y": training.SIZE[1],
            "z": training.SIZE[2],
        },
        "voxels": {},
    }

    for x in range(training.SIZE[0]):
        for y in range(training.SIZE[1]):
            for z in range(training.SIZE[2]):
                # Apply offset
                cur_pos = (x, y, z)
                offset_pos = vec.add(cur_pos, offset)
                offset_pos = (offset_pos[0] % size, offset_pos[1] % size, offset_pos[2] % size)

                # Build lattice structure
                if offset_pos[0] == 0 and offset_pos[1] == 0 and offset_pos[2] == 1:
                    data["voxels"][vec.ttos(cur_pos)] = color1
                else:
                    c = 0
                    for pos in offset_pos:
                        if pos == 0:
                            c += 1
                    if c >= 2:
                        data["voxels"][vec.ttos(cur_pos)] = color2

    # Save JSON file
    output_filename = f'training/json/sorpok{i}.json'
    with open(output_filename, 'w') as output_file:
        json.dump(data, output_file, indent=2)


In [19]:
# Expand training examples so that each voxel color appears a minimum number of times
import json
import random
import copy
import vector
import os
MIN_EACH_COLOR = 10
MAX_ITERATIONS = 100
FOLDER = "training/json/"

# Delete existing expanded files
exp_files = os.listdir(FOLDER)
for item in exp_files:
    if "_exp_" in item:
        os.remove(os.path.join(FOLDER, item))

color_counts = []
for _ in range(255):
    color_counts.append(MIN_EACH_COLOR)

def random_color_shift():
    def rv():
        return random.random() - 0.5
    m = rv() + rv() + rv() + rv() + rv() + rv()
    return m * 128

def clamp_color(color):
    def cv(c):
        r = max(min(c, 255), 0)
        return int(r/5) * 5

    return (
        cv(color[0]),
        cv(color[1]),
        cv(color[2]),
    )

def expand_example(example, filename, color_counts):
    filename_split = filename.split('.')
    new_filename = f"_exp_{filename_split[0]}_{str(int(random.random() * 99999999999))}.{filename_split[1]}"

    # Choose a random rgb shift
    rs = random_color_shift()
    gs = random_color_shift()
    bs = random_color_shift()

    # Shift the color by that amount
    copied_example = copy.deepcopy(example)
    with open("data/palette.json", 'r') as palette_file:
        with open("data/palette_map.json", 'r') as palette_map_file:
            palette = json.load(palette_file)
            palette_map = json.load(palette_map_file)
            
            for voxel in copied_example["voxels"]:
                color_index = copied_example["voxels"][voxel]
                old_color = palette["colors"][color_index]
                new_color = vector.add(old_color, (rs, gs, bs))
                new_color = clamp_color(new_color)
                new_color_index = palette_map[vector.ttos(new_color)]
                copied_example["voxels"][voxel] = new_color_index
    
    # Check if this example will actually add any new colors
    changed = check_color_counts(color_counts, copied_example, False)

    # If changes were made, create this example
    if changed:
        # Update color counts
        check_color_counts(color_counts, copied_example, True)

        # Save file
        with open(FOLDER + new_filename, 'w') as output_file:
            json.dump(copied_example, output_file, indent=2)

# Check if this example will change the color counts. Also used to edit color counts from an example
def check_color_counts(color_counts, example, edit):
    def ctoi(c):
        if c == 0 or c == 1:
            return 0
        return c-1

    ret = False
    for voxel_key in example["voxels"]:
        voxel_color = example["voxels"][voxel_key]
        if color_counts[ctoi(voxel_color)] > 0:
            if edit:
                color_counts[ctoi(voxel_color)] -= 1
            ret = True
    return ret

# Iterate process
for i in range(MAX_ITERATIONS):
    print(f"Iteration {i+1}")

    # Go through the files and make n attempts to expand the example
    filenames = os.listdir('training/json')
    filenames = list(filter(lambda f : not ("_exp" in f), filenames))
    for filename in filenames:
        with open(FOLDER + filename, 'r') as expanded_file:
            example = json.load(expanded_file)
            if i == 0:
                # Set initial color counts
                check_color_counts(color_counts, example, True)
            expand_example(example, filename, color_counts)

    # Check if we have enough of every color, and exit if we do
    colors_not_filled = len(list(filter(lambda c: c > 0, color_counts)))
    print(f"{colors_not_filled} colors remaining")
    if colors_not_filled == 0:
        break
        



Iteration 1
199 colors remaining
Iteration 2
178 colors remaining
Iteration 3
159 colors remaining
Iteration 4
144 colors remaining
Iteration 5
137 colors remaining
Iteration 6
134 colors remaining
Iteration 7
129 colors remaining
Iteration 8
123 colors remaining
Iteration 9
113 colors remaining
Iteration 10
111 colors remaining
Iteration 11
107 colors remaining
Iteration 12
106 colors remaining
Iteration 13
102 colors remaining
Iteration 14
99 colors remaining
Iteration 15
97 colors remaining
Iteration 16
89 colors remaining
Iteration 17
87 colors remaining
Iteration 18
86 colors remaining
Iteration 19
85 colors remaining
Iteration 20
83 colors remaining
Iteration 21
82 colors remaining
Iteration 22
80 colors remaining
Iteration 23
78 colors remaining
Iteration 24
74 colors remaining
Iteration 25
73 colors remaining
Iteration 26
73 colors remaining
Iteration 27
70 colors remaining
Iteration 28
67 colors remaining
Iteration 29
67 colors remaining
Iteration 30
66 colors remaining
Iterat